# LSH算法

**Reference**

https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/

[数据来源](http://www.kaggle.com/benhamner/exploring-the-nips-papers)

[datasketch包](http://ekzhu.com/datasketch/documentation.html)

[minHash(最小哈希)和LSH(局部敏感哈希)概述](https://blog.csdn.net/liujan511536/article/details/47729721)


本节的主要目标是通过使用LSH快速查询所有已知的文章，然后作出推荐。

In [1]:
# 导入各种包
import numpy as np
import pandas as pd
import re
import time
# ! pip install datasketch
from datasketch import MinHash, MinHashLSH


## 数据预处理

首先，我们要进行简要的shingles, 通常包含以下几个步骤

1. 删掉所有标点符号
1. 将所有单词小写表示
1. 通过用空格来分隔开每个单词

通常为了更好的效果，会使用一些自然语言处理的语料库像NLTK或spaCy来处理一些常见单词或非有效词，本节暂时不考虑。

In [2]:
# 定义预处理函数
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text) 
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [3]:
text = 'The devil went down to Georgia'
print('分词后：', preprocess(text))

分词后： ['the', 'devil', 'went', 'down', 'to', 'georgia']


## 选择排序数
本节以排序数128为例子

In [4]:
permutation = 128

## 定义一个minhash函数

In [5]:
def myMinHash(tokens, perms):
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
    return m



## 评估查询

我们将首先下载包含所有会议论文的CSV，并创建一个新字段，将标题和摘要合并到一个字段中，这样我们就可以同时使用标题和摘要来构建shingles。

最后，我们可以查询任何文本字符串，如标题或一般主题，本节使用相似度高的标题用作推荐。

In [6]:
db = pd.read_csv('papers.csv')

#数据预处理
db['data']=db['title'].apply(preprocess)

#为每一行创建一个minHash，并将其保存到列mHash中
db['mhash'] = db['data'].apply(myMinHash, args=(permutation,))

#创建一个名为lsh的MinHashLSH对象。
lsh = MinHashLSH(num_perm=permutation, params=[32,4])
db.apply(lambda x: lsh.insert(x['id'], x['mhash']), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
7236    None
7237    None
7238    None
7239    None
7240    None
Length: 7241, dtype: object

In [7]:

# 创建一些与论文相似的标题。看看这个算法是否能够推荐相似的论文题目
titles = {'Self-Organization of Associative Database and Its Applications',
          'Self-Organization of Associative Database and  Applications',
          'Self-Organization of Associative Database ants Applications',
          'Self-Organization of Associative Database d Its Applications',
          'Self-Organization of Associative Datase and Its Applications',
          'Self-Organization of and Its Applications'
        }

for title in titles:
    a = preprocess(title)
    mhash = myMinHash(a, permutation)

    u = lsh.query(mhash)
    
    idx_array = np.array(u)
    result = db.loc[db['id'].isin(idx_array)]['title']
    
    print('\n\标题', title, '\t 匹配数:', len(result), '\n---------------------------')
    for i in result:
        print(i)
    


\标题 Self-Organization of Associative Database ants Applications 	 匹配数: 1 
---------------------------
Self-Organization of Associative Database and Its Applications

\标题 Self-Organization of Associative Database d Its Applications 	 匹配数: 1 
---------------------------
Self-Organization of Associative Database and Its Applications

\标题 Self-Organization of and Its Applications 	 匹配数: 25 
---------------------------
Self-Organization of Associative Database and Its Applications
The Geometry of Eye Rotations and Listing's Law
Minimax and Hamiltonian Dynamics of Excitatory-Inhibitory Networks
Stationarity and Stability of Autoregressive Neural Network Processes
Synergy and Redundancy among Brain Cells of Behaving Monkeys
Scale Mixtures of Gaussians and the Statistics of Natural Images
Model Complexity, Goodness of Fit and Diminishing Returns
A Comparison of Image Processing Techniques for Visual Speech Recognition Applications
Convergence of Optimistic and Incremental Q-Learning
A Model o

这效果看着还行。